In [109]:
import sys
import notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())
import datasets
import math
import os
import pandas as pd
import numpy as np
import roc_auc
import estimator_kmeans as kmeans
import estimator_knn as knn
import estimator_nn as nn
import estimator_rssibased as rssie
from datetime import datetime
import json
from IPython.display import display, HTML

In [14]:
# data loader
ds = datasets.load('data/raw/0[12]_[01][123]_0[1234]*_*')

path = data/raw/0[12]_[01][123]_0[1234]*_*
data/raw/01_01_01_4F実験室_XperiaZ3_胸ポケット_裏上_正常_まっすぐ帰宅
..............................
data/raw/01_01_02_4F実験室_XperiaZ3_カバン_裏上_正常_まっすぐ帰宅
...x..........................
data/raw/01_01_03_4F実験室_XperiaZ3_胸ポケット_裏上_正常_5秒後まっすぐ帰宅
.........................x....
data/raw/01_01_04_4F実験室_XperiaZ3_カバン_裏上_正常_5秒後まっすぐ帰宅
.....................x........
data/raw/01_02_01_4F実験室_XperiaZ3_胸ポケット_裏上_異常_まっすぐ外出
..............................
data/raw/01_02_02_4F実験室_XperiaZ3_ズボン_裏上_異常_まっすぐ外出
..............................
data/raw/01_02_03_4F実験室_XperiaZ3_カバン_裏上_異常_まっすぐ外出
..............................
data/raw/01_03_01_4F実験室_XperiaZ3_胸ポケット_裏上_異常_まっすぐ帰宅
..............................
data/raw/01_03_02_4F実験室_XperiaZ3_ズボン_裏上_異常_まっすぐ帰宅
..............................
data/raw/01_03_03_4F実験室_XperiaZ3_カバン_裏上_異常_まっすぐ帰宅
..............................
data/raw/01_11_01_エネマネハウス_XperiaZ3_胸ポケット_裏上_正常_まっすぐ帰宅
..............................
data/raw/01_11_02_エネマネハウス_XperiaZ3_カバン_裏上_正常_まっす

In [20]:
# general estimator test
def eval_estimator(
    model,
    sensor_type = ['rssi.a','rssi.b', ['linear_accel[0]','linear_accel[1]','linear_accel[2]']],
    n_record = 3,
    base = '01_11_01',
    normal = '^01_11',
    anomaly = '^01_1[23]',
    ms_interval = 20,
    ma_window = 3,
    n_train = 3,
    normalize = True):
    
    # recalc input
    drop_interval = int(ms_interval / 20)
    
    # get data
    dfl_t = datasets.get_data(ds, title=base, before=n_record,
                              column=sensor_type, drop_interval=drop_interval)[:(n_train + 1)]
    dfl_n = datasets.get_data(ds, title=normal, before=n_record,
                              column=sensor_type, drop_interval=drop_interval)
    dfl_o = datasets.get_data(ds, title=anomaly, before=n_record,
                              column=sensor_type, drop_interval=drop_interval)
    
    # moving average
    dfl_t = datasets.moving_average(dfl_t, window=ma_window, min_periods=ma_window)
    dfl_n = datasets.moving_average(dfl_n, window=ma_window, min_periods=ma_window)
    dfl_o = datasets.moving_average(dfl_o, window=ma_window, min_periods=ma_window)

    # normalize
    if normalize == True:
        dfl_t_n = datasets.normalize_by_base_data(dfl_t, dfl_t, sensor_type)
        dfl_n_n = datasets.normalize_by_base_data(dfl_t, dfl_n, sensor_type)
        dfl_o_n = datasets.normalize_by_base_data(dfl_t, dfl_o, sensor_type)

    # get numpy array
    data_2d_t = [df.as_matrix() for df in dfl_t_n]
    data_2d_n = [df.as_matrix() for df in dfl_n_n]
    data_2d_o = [df.as_matrix() for df in dfl_o_n]

    # to list of numpy.array
    data_t = [d.ravel() for d in data_2d_t]
    data_n = [d.ravel() for d in data_2d_n]
    data_o = [d.ravel() for d in data_2d_o]
    
    # get auc score
    model.fit(data_t)
    score_n = model.decision_function(data_n)
    score_o = model.decision_function(data_o)
    auc = roc_auc.get_auc_from_normal_outlier(score_n, score_o, graph=False)
    
    return auc

#eval_estimator()

In [108]:
# grid search implementation (under test)
def _get_estimator_models():
    models = []
    
    # k-means based estimator
    range_n_clusters = np.arange(1, 10, 2)
    mesh_data = np.meshgrid(range_n_clusters)
    for n_clusters, in zip(mesh_data[0].ravel()):
        models.append(kmeans.EstimatorKmeans(n_clusters=n_clusters))
    
    # knn based estimator
    range_n_neighbors = np.arange(1, 10, 2)
    mesh_data = np.meshgrid(range_n_neighbors)
    for n_neighbors, in zip(mesh_data[0].ravel()):
        models.append(knn.EstimatorKNN(n_neighbors=n_neighbors))
    
    # nn based estimator
    range_n_training_epochs = np.array([100])
    range_n_hidden_nodes = np.array([4, 8, 16, 32])
    mesh_data = np.meshgrid(range_n_training_epochs, range_n_hidden_nodes)
    for n_training_epochs, n_hidden_nodes in zip(mesh_data[0].ravel(), mesh_data[1].ravel()):
        models.append(nn.EstimatorNN(num_of_hidden_nodes=n_hidden_nodes,
                                     num_of_training_epochs=n_training_epochs))
    
    # rssi based estimator
    models.append(rssie.EstimatorRssiBased())
    
    return models

def _get_grid_test_case(models):
    # parameters other than algorithm specific
    sensor_master = [
        ['rssi.a', 'rssi.b'],
        ['rssi.a', 'rssi.b', ['acceleration.x', 'acceleration.y', 'acceleration.z']],
        ['rssi.a', 'rssi.b', ['gyro.rotationRate.x', 'gyro.rotationRate.y', 'gyro.rotationRate.z']],
        ['rssi.a', 'rssi.b', ['magneticField.x', 'magneticField.y', 'magneticField.z']],
        ['rssi.a', 'rssi.b', ['attitude.roll', 'attitude.pitch', 'attitude.yaw']],
        ['rssi.a', 'rssi.b', ['rotationRate.x', 'rotationRate.y', 'rotationRate.z']],
        ['rssi.a', 'rssi.b', ['gravity.x', 'gravity.y', 'gravity.z']],
        ['rssi.a', 'rssi.b', ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']],
    ]
    sensor_master_s = [json.dumps(s) for s in sensor_master] # to be combinationable

    range_n_record = np.array([1, 5, 20])
    range_n_train = np.array([1, 5, 10, 20, 100])
    range_ms_interval = np.array([20, 60, 100])
    range_ma_window = np.arange(1, 4, 2)
    
    mesh_data = np.meshgrid(range(len(models)), range_n_record, range_n_train,
                            range_ms_interval, range_ma_window, range(len(sensor_master_s)))

    # create grid test case
    test_case = []
    for model_idx, n_record, n_train, ms_interval, ma_window, sensor_idx in zip(mesh_data[0].ravel(), mesh_data[1].ravel(), mesh_data[2].ravel(),
               mesh_data[3].ravel(), mesh_data[4].ravel(), mesh_data[5].ravel()):
        model = models[model_idx]
                            
        # save test case and result
        _t = {'model_idx': model_idx, 'n_record': n_record, 'n_train': n_train,
              'ms_interval': ms_interval, 'ma_window': ma_window, 'sensor_type': sensor_master_s[sensor_idx]}
        _t.update(model.get_params())
        _t['type'] = model.get_type()
        test_case.append(_t)
    
    # create df for test case
    df = pd.DataFrame(test_case)
    return df

def _remove_unavailable_test_case(df):
    # remove unavailable test case
    df = df[~(df['n_train'] < df['n_neighbors'])]
    df = df[~(df['n_train'] < df['n_clusters'])]
    df = df[~((df['type'] == 'rssi_based') & (df['sensor_type'].apply(lambda x: len(json.loads(x)) > 2)))]
    df = df[~(df['n_record'] - df['ma_window'] < 0)]
    return df

def _run_test(df, models):
    # set the data title
    base = '02_11_01'
    normal = '^02_11'
    anomaly = '^02_1[23]'
    
    csv_fname = 'test_record_%s__%s__%s.csv' % (base, normal, anomaly) 

    # if auc is already computed in some test case, merge the result. 
    if os.path.exists(csv_fname):
        df_past = read_csv(csv_fname)
        keys = list(df.columns.values)
        df = pd.merge(df, df_past, on=keys, how='outer')
    else:
        df['auc'] = np.nan
    
    imcomplete_test_case = len([x for x in df['auc'].values if np.isnan(x)])
    print('total test case: %d' % (len(df.index)))
    print('imcomplete test case: %d' %(imcomplete_test_case))
    
    # run test
    for i, (k, t) in enumerate(df[df['auc'].isnull()].iterrows()):
        if i % 10 == 0:
            print("%d / %d at %s"%(i+1, imcomplete_test_case, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
            df.to_csv(csv_fname )

        auc = eval_estimator(models[t['model_idx']], base=base, normal=normal, anomaly=anomaly,
                             n_train=t['n_train'], sensor_type=json.loads(t['sensor_type']), n_record=t['n_record'],
                             ms_interval=t['ms_interval'], ma_window=t['ma_window'])
        df.loc[[k], 'auc'] = auc
    
    df.to_csv(csv_fname )
    return df

def test():
    models = _get_estimator_models()
    df = _get_grid_test_case(models)
    df = _remove_unavailable_test_case(df)
    df = _run_test(df, models)
    
    return df

test()

total test case: 7035
imcomplete test case: 7035
1 / 7035
11 / 7035
21 / 7035
31 / 7035
41 / 7035
51 / 7035
61 / 7035
71 / 7035
81 / 7035
91 / 7035


KeyboardInterrupt: 

In [55]:
def read_csv(fname):
    df = pd.read_csv(fname, index_col=0)
    return df
#read_csv()